In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML
from scipy.integrate import quad

%matplotlib inline 

# Exponential current-based synapses

Here, we want look at current-based synapses. We assume that the current after the arrival of a spike can simply modeled by instantaneous current jump followed by an exponential decay. Thus, the current dynamics are governed by the differential equation

\begin{equation} \label{eq: deq I}
\frac{dI(t)}{dt}=-\frac{1}{\tau}I(t)+ h s(t),
\end{equation}

where $\tau$ is the intrinsic synaptic time constant, $h$ is the synaptic efficacy and $s(t)$ represents the incoming spikes 

\begin{equation} \label{eq: spike train}
s(t)=\sum_{i}\delta(t-t_{i}).
\end{equation}


## Deterministic description

A spike arriving at $t_{i}$ will cause a jump of the current by $h$ followed by an exponential decay to $0$ due to the leek term. The efficacy $h$ can be positive in case of an excitatory or negative in case of inhibitory synapse. The homogeneous part of \ref{eq: deq I} can be written as 

\begin{equation}
O\,I(t)=0,
\end{equation}


with the differential operator $O$ given by $0=\tau\frac{d}{dt}+1$. For the initial condition $I(0)=I_0$, the homogeneous solution is given by     

\begin{equation}
I(t)=I_0 e^{-\frac{t}{\tau}}
\end{equation}

To solve \ref{eq: deq I} with inhomogeneity, we can make use of the fact that $G(t)=e^{-\frac{t}{\tau}}\theta(t)$ is the Green's function of the differential operator $O$, i.e. 

\begin{equation} \label{eq: Green's fcn}
O\,G(t)=\delta(t)
\end{equation}

Thus, convolving the Green's function with inhomogeneity $hs(t')$ yields the particular solution   

\begin{equation} \label{eq: solution I}
I(t)=\int_{0}^{\infty}dt'\,G(t-t') hs(t').
\end{equation}

This can be seen if we insert \ref{eq: solution I} in \ref{eq: deq I} and use \ref{eq: Green's fcn}. Thus, for $s(t)$ given by \ref{eq: spike train} we arrive at 

\begin{equation}
I(t)=h\sum_{i=1}\int_{0}^{t}dt'\,\frac{1}{\tau}e^{-\frac{t-t'}{\tau}}\,\delta(t'-t_{i}).
\end{equation}

The delta-distribution gets rid of the integral leading to   

\begin{equation}
I(t) = h\sum_{i=1}e^{-\frac{t-t_{i}}{\tau}}.
\end{equation}


Adding the homogeneous solution, we arrive at

\begin{equation}
I(t) = I_0 e^{-\frac{t}{\tau}} + h\sum_{i=1}e^{-\frac{t-t_{i}}{\tau}}.
\end{equation}

Thus, we find that $I(t)$ is given by the superposition of individual exponentially decaying current jumps invoked by the different spikes. We now look at the time evolution of the current assuming that the incoming spikes can be modeled by homogeneous Poisson process with rate $\lambda$. 

In [ ]:
def poisson_spike_train(lam, T):
    '''Draw random spike times for a Poisson process with constant rate within the given time interval 
    :param lam: rate
    :param T: recording time  
    :return spike_train: spike times 
    '''
    n = np.random.poisson(T*lam)
    spike_train = np.random.uniform(0, T, n)

    return spike_train

def exp_decaying_current(t, spike_train, tau, h):
    '''Caculate current trace at time points t for incoming spike train
    :param t: time points
    :param spike_train: spike_times   
    :param tau: synaptic time constant
    :param h: synaptic efficacy
    :return I: current trace'''
    
    I = np.zeros_like(t)
    
    for t_spike in spike_train:
                    
        H_t = 0.5*(1 + np.sign(t-t_spike)) # heaviside function centered around spike time                    
        I += h*np.exp(-(t-t_spike)/tau)*H_t # exponential decay 

    return I 

#------------------------------------------------------------------------------ 
# Plot Synaptic current for different input rates  
T = 50

lam0 = 0.1
lam1 = 1.
lam2 = 10.

spike_train0 = poisson_spike_train(lam0, T)
spike_train1 = poisson_spike_train(lam1, T)
spike_train2 = poisson_spike_train(lam2, T)


tau = 1. # intrinsic synapse time constant
h = 1. # efficacy 

delta_t = 0.001*tau # time step

t = np.linspace(0, T, int(np.ceil(T/delta_t))) # time points 

# calculate current traces for different input rates
I0 = exp_decaying_current(t, spike_train0, tau, h) 
I1 = exp_decaying_current(t, spike_train1, tau, h) 
I2 = exp_decaying_current(t, spike_train2, tau, h) 

plt.figure(figsize = (14, 6))
gs = plt.GridSpec(3, 1)

ax0 = plt.subplot(gs[0]) 
ax0.plot(t, I0, '-',  color = 'k')
ax0.plot(spike_train0, np.zeros_like(spike_train0), 'o', color = 'blue')

plt.ylabel('$I(t)$', fontsize = 18)

ax1 = plt.subplot(gs[1]) 
ax1.plot(t, I1, '-',  color = 'k')
ax1.plot(spike_train1, np.zeros_like(spike_train1), 'o', color = 'blue')

plt.ylabel('$I(t)$', fontsize = 18)

ax2 = plt.subplot(gs[2]) 
ax2.plot(t, I2, '-',  color = 'k')
ax2.plot(spike_train2, np.zeros_like(spike_train2), 'o', color = 'blue')

plt.xlabel('$t$', fontsize = 18)
plt.ylabel('$I(t)$', fontsize = 18)

plt.show()

## Stochastic description 

If the rate $\lambda$ is large compared the to relaxation time constant $\tau$ then the current will fluctuate around some stationary baseline after some initial transient. This can be seen in panel three of the above figure. 

Instead of solving

\begin{equation} \label{eq: deq I}
\tau\frac{dI(t)}{dt}=-I(t)+h\cdot s(t),
\end{equation}

for one specific realization of the spike train $s(t)$, we aim for a stochastic description which relates the statistics of the current trace to the statistics of the Poisson process. To time evolution of $I(t)$, we need to determine the probability density $\rho(I, t)$ which tells how likely it is to observe the current $I$ at time $t$. It can expressed terms of conditional probability $\rho(I,t|J,s)$ which leads to the Chapman-Kolmogorov equation    

\begin{equation}
\rho(I,t)=\int dJ\,\rho(I,t|J,s)\rho(J,s)
\end{equation}

The latter can be rewritten as an differential equation, which leads in second order approximation to the Fokker-Planck equation 

\begin{equation} \label{fpe}
\frac{\partial \rho(I,t)}{\partial t}=\left(-\frac{\partial}{\partial I}A_{1}(I,t)+ \frac{\partial^{2}}{\partial I^{2}}A_{2}(I,t)\right)\rho(I,t).
\end{equation}

Defining the probability flux 

\begin{equation} \label{probability flux}
\Phi(I,t)=\left (A_{1}(I,t)-\frac{\partial}{\partial I}A_{2}(I,t)\right)\rho(I,t).
\end{equation}

the Fokker-Planck equation can be written as continuity equation 

\begin{equation}
\frac{\partial \rho(I,t)}{\partial t}=-\frac{\partial \Phi(I,t)}{\partial I}
\end{equation}

Here, $A_{1}$ is the drift coefficient and $A_{2}$ the diffusion coefficient which are defined as  

\begin{equation} \label{infinitesimal moment}
A_{n}(I,t)=\lim_{\delta t \rightarrow 0}\frac{a_{n}(I,\delta t,\,t)}{n! \delta t},
\end{equation}

where $a_{n}(I,t+\delta t,\,t)$ is the conditional moment defined as 

\begin{equation} \label{conditional moment}
a_{n}(I,\delta t,\,t)=\int d\Delta\,\Delta^{n}\rho(I+\Delta,t+\delta t|I,t)
\end{equation}

Here, $\rho(I+\Delta,t+\delta t|I,t)$ is the conditional probability that the current changed by $\Delta$ after a time step $\delta t$ assuming that the value of the current was $I$ at the $t$ beforehand. The above equation offers an intuitive explanation for the coefficients $a_{1}$ and $a_{2}$:
-  $a_{1}$ can be understood as the expected  drift averaged over all trajectories which start at time $t$ at $I$ step and than evolve from $t$ to $\delta t$ 


-  $a_{2}$ is the second moment, i.e. $a_{2}-a_{1}^2$ measures how the distribution of $\Delta$, and therefore $\rho(I,t)$ spreads with time

Will now determine the first two infinitesimal moments \ref{infinitesimal moment} for \ref{eq: deq I} under the assumption that $s(t)$ is Poisson distributed. Since ultimately, we are interested in the limes $\delta t \rightarrow 0$, we only need to consider two cases:

1. There is exactly one incoming spike in the interval $\delta t$, which happens with probability $p_{1}=\lambda \delta t$
2.  There is no incoming spike in the interval $\delta t$, which happens with probability $p_{2}=1-p_{1}$

For both cases, we determine the time-evolution of the current in the infinitesimal step $delta t$ which only depends on the current $I$ at the initial time point $t. For the first case, we get  

\begin{equation}
\Delta_{1}=I(t+\delta t)-I(t)=-\frac{I(t)}{\tau}\delta t+h.
\end{equation}

For the second case, we get

\begin{equation}
\Delta_{1}=I(t+\delta t)-I(t)=-\frac{I(t)}{\tau}\delta t.
\end{equation}

Thus, the conditional probability $\rho(I+\Delta,t+\delta t|I,t)$ in \ref{conditional moment} has only two discrete values so that the integral can be replaced by a sum

\begin{equation}
a_{n}(I,\delta t,t)=\Delta_{1}^{n}p_{1}+\Delta_{2}^{n}p_{2}
\end{equation}

Hence, the first two infinitesimal moments are given by 

\begin{equation}
a_{1}(I,\delta t,t)=\left(-\frac{I(t)}{\tau} + h\lambda\right)\delta t + \mathcal{O}(\delta t^{2}),
\end{equation}

and 

\begin{equation}
a_{2}(I,\delta t,t) = h^{2} \lambda \delta t.
\end{equation}

Inserting into \ref{infinitesimal moment} yields the first two infinitesimal moments  

\begin{equation}
A_{1}(I,t)= -\frac{1}{\tau}(I(t) - \mu),
\end{equation}

with expected baseline current $\mu=\tau \lambda h$ and  

\begin{equation} \label{eq: A_2}
A_{2}(I,t)= \frac{\sigma^{2}}{\tau},
\end{equation}

with $\sigma^{2}=\tau \lambda h^2$/2. Inserting into \ref{probability flux} we arrive at

\begin{equation} \label{eq: fpe 2}
\frac{\partial \rho(I,t)}{\partial t} =-\frac{\partial \Phi(I,t)}{ \partial I},
\end{equation}

with the probability flux $\Phi(I,t)$ given by

\begin{equation} \label{eq: probability flux 2}
\Phi(I,t) = -\left(\frac{1}{\tau}(I(t) - \mu) + \frac{\sigma^{2} }{\tau}\frac{\partial}{ \partial I} \right)\rho(I,t).
\end{equation}.

Note that we would obtain the same result if we replace the Poisson input by Gaussian white noise with mean $\mu$ and variance $\sigma^2$. Hence, in second order approximation, we can effectively replace \ref{eq: deq I} by 

\begin{equation} \label{eq: langevin}
\frac{dI(t)}{dt}=-\frac{1}{\tau}(I(t)-\mu)+\xi(t),
\end{equation}.

where $\xi(t)$ is a Gaussian white noise with zero mean $\left< \xi(t) \right> = 0$ and delta-shaped autocorrelation  $\left< \xi(t) \xi(t')\right> = \tilde{\sigma}^2 \delta(t-t')$. The noise strength $\tilde{\sigma}^2=2\sigma^2/\tau$ has to be chosen such that we recover the same diffusion coefficient as in \ref{eq: probability flux 2}. Eq. \ref{eq: langevin} is a linear Langevin equation from which the time evolution of the moments of $I(t)$ can be directly calculated for arbitrary initial conditions.

Let's assume that for each trial, the current is zero at beginning of the recording , i.e. the initial condition is $I(0)=0$. In this case, the solution to \ref{eq: langevin} is given by 

\begin{equation} 
I(t)=\mu \left(1-e^{-\frac{t}{\tau}}\right) + \int_{0}^{t}dt'e^{-\frac{t-t'}{\tau}}\xi(t')
\end{equation}.

Using $\left< \xi(t) \right> = 0$, we obtain 

\begin{equation} 
\left< I(t)\right> = \mu \left(1-e^{-\frac{t}{\tau}}\right),
\end{equation}.

which is simply the solution we would have obtained if we neglected the noise term in \ref{eq: langevin} from the beginning on. For $t/\tau \gg 1$, we obtain the stationary limit 

\begin{equation} \label{stationary expectation value}
\left< I(t)\right> = \mu.
\end{equation}.
 
The correlation function $a(t_1, t_2) = \left< I(t_{1})I(t_{2})\right>$ is given by  

\begin{equation} 
a(t_1, t_2) = \mu^{2}\left(1-e^{-\frac{t_{1}}{\tau}}-e^{-\frac{t_{2}}{\tau}}+e^{-\frac{t_{1}+t_{2}}{\tau}}\right)+\frac{\tau \tilde{\sigma}^2}{2}\left(e^{-\frac{\left|t_{1}-t_{2}\right|}{\tau}}-e^{-\frac{t_{1}+t_{2}}{\tau}}\right)
\end{equation}.

For $t_{1}/\tau \gg 1$ and $t_{2}/\tau \gg 1$, we obtain the stationary limit 

\begin{equation} 
a_{\infty}(\left|t_{1}-t_{2}\right|) = \mu^{2} + \frac{\tau \tilde{\sigma}^2}{2}e^{\frac{\left|t_{1}-t_{2}\right|}{\tau}}
\end{equation},

which depends only on the time difference $\left|t_{1}-t_{2}\right|$. Hence, in the stationary limit, we obtain the following epxression fir the centralized zero lag autocorrelation function     

\begin{equation} \label{eq: stationary zero lag cf}
a_{\infty}(0) - \mu^2 =\frac{\tau \tilde{\sigma}^2}{2} = \sigma^2.
\end{equation}

Alternatively, we can derive the stationary current distribution $\rho(I,t)$ from the Fokker-Planck equation. This allows us caculate moments of $I(t)$ in the stationary limit. In the stationary case, the l.h.s. of \ref{eq: fpe 2} must be zero, i.e. the probability flux \ref{eq: probability flux 2} must be a constant. For natural boundary conditions, we can further conclude that the flux must be zero and the current distribution is given by a Gaussian distribution with mean $\mu=\tau \lambda h$ and variance $\sigma^2 = \tau \lambda h^2$ 

\begin{equation} \label{eq: stationary rho}
\rho(I) = \frac{1}{\sqrt{2\pi}\sigma}e^{-\frac{(I-\mu)^{2}}{2 \sigma^{2}}}.
\end{equation}

Note that mean and variance of the Gaussian distribution are equivalent to the stationary limit of the expectation value \ref{stationary expectation value} and the zero lag correlation function \ref{eq: stationary zero lag cf} which we obtained directly from the Langevin equation. 

We will now generate an ensemble of trials and check if they are centered around $\mu$ and if their fluctuations stay in the range of $\mu \pm \tau  \sigma^2$.   

In [ ]:
def generate_exp_decaying_current_trials(N, T, t, tau, h, lam, I0):
    '''Generate N current traces for randomly drawn Poisson spike trains
    :param N: number of trials
    :param t: time points 
    :param tau: synaptic time constant
    :param h: synaptic efficacy
    :param I0: initial condition 
    :return I_mat: N x len(t) matrix with current traces
    '''

    I_mat = np.tile(np.exp(-t/tau), (N,1))

    if np.isscalar(I0): # fixed initial value
        I_mat = I0*I_mat         
    else: # initial values are drawn from distribution
        for i, I0_i in enumerate(I0):
            I_mat[i,:] *= I0_i
                       
    for i in xrange(N):
        
        spike_train = poisson_spike_train(lam, T)
        I_mat[i, :] += exp_decaying_current(t, spike_train, tau, h)

    return I_mat

#------------------------------------------------------------------------------ 
# parameter

lam = 10. # rate
tau = 2 # intrinsic synapse time constant
h = 0.1 # efficacy 

N = 1000 # number of trials
T = 10 # recording time 

dt = 0.001*tau # time step
t = np.arange(0, T + dt, dt) # time points 
M = len(t) # number of time points

#------------------------------------------------------------------------------ 
# diffusion approximation

mu = tau*lam*h 
sig2 = 0.5*tau*lam*h**2

#------------------------------------------------------------------------------ 
# generate current trials

# sharp initial condition I(0)=0  
I0 = 0

I_mat_0 = generate_exp_decaying_current_trials(N, T, t, tau, h, lam, I0)

# draw initial values from stationary distribution
I0 = np.random.normal(mu, np.sqrt(sig2), N)

I_mat_1 = generate_exp_decaying_current_trials(N, T, t, tau, h, lam, I0)

#------------------------------------------------------------------------------ 
# plotting

plt.figure(figsize = (14, 10))

gs = plt.GridSpec(2,1)

ax0 = plt.subplot(gs[0])

for I in I_mat_0:
    
    ax0.plot(t, I, '-', linewidth = 0.1)  

y_max = max(max(I_mat_0.flatten()), max(I_mat_1.flatten())) 

plt.hlines(mu, 0, t[-1], colors = 'k', linestyles = '-', label = r'$\mu$')
plt.hlines(mu - 2*np.sqrt(sig2), 0, t[-1], colors = 'k', linestyles = '--', label = r'$\mu - 2\sigma$')
plt.hlines(mu + 2*np.sqrt(sig2), 0, t[-1], colors = 'k', linestyles = '--', label = r'$\mu + 2\sigma$')

ax0.set_title(r'Initial condition $I(0)=0$', fontsize = 18)    
plt.xlim(0, t[-1])
plt.ylim(0, y_max)
plt.legend(loc = 'upper right', fontsize = 16)

plt.ylabel('$I(t)$', fontsize = 18)

ax1 = plt.subplot(gs[1])

for I in I_mat_1:
    
    ax1.plot(t, I, '-', linewidth = 0.1)  

plt.hlines(mu, 0, t[-1], colors = 'k', linestyles = '-', label = r'$\mu$')
plt.hlines(mu - 2*np.sqrt(sig2), 0, t[-1], colors = 'k', linestyles = '--', label = r'$\mu - 2\sigma$')
plt.hlines(mu + 2*np.sqrt(sig2), 0, t[-1], colors = 'k', linestyles = '--', label = r'$\mu + 2\sigma$')
        
plt.xlim(0, t[-1])
plt.ylim(0, y_max)
    
ax1.set_title(r'Initial condition drawn from stationary distribution', fontsize = 18)    
plt.xlabel(r'$t$', fontsize = 18)
plt.ylabel(r'$I(t)$', fontsize = 18)
plt.legend(loc = 'upper right', fontsize = 16)

plt.show()

To estimate the stationary distribution, we will make a histogram out of the different trials. Since, we assume a stationary distribution we can simply pool all values of $I(t)$ over entire recording time. 

In [ ]:
#------------------------------------------------------------------------------ 
# Stationary distribution

I_arr = I_mat_1.flatten() # flatten time resolved current trials

plt.figure(figsize = (14, 6))

plt.hist(I_arr, 100, density = True)

plt.xlabel(r'$I$', fontsize = 18)
plt.ylabel(r'$\rho(I)$', fontsize = 18)

# compare to stationary distribution obtained from Fokker-Planck equation

mu = tau*lam*h 
sig2 = 0.5*tau*lam*h**2

I = np.linspace(min(I_arr), max(I_arr), 1000) 
rho = 1./np.sqrt(2*np.pi*sig2)*np.exp(-0.5*(I-mu)**2/sig2)

plt.plot(I, rho, '-', color = 'red', linewidth = 2, label = r'$\rho(I)$ stationary')
plt.legend(loc = 'upper right', fontsize = 16)
plt.show()

## Nonhomogeneous input rate

Here we want to investigate if our result for the stationary distribution are still valid in case of a time varying firing rate $\lambda(t)$. Note that $\rho(I,t)$ must depend on time in the stationary limit due to the time dependence of rate. However, assuming that $\lambda$ varies slowly compared to the synaptic time constant $\tau$, we may simply inlcude the time dependence into the drift and diffusion coefficient derived beforehand

\begin{equation}
A_{1}(I,t)= -\frac{1}{\tau}(I(t) - \mu(t)),
\end{equation}

with time dependent expected baseline current $\mu(t)= \tau h\lambda(t)$ and  

\begin{equation} \label{eq: A_2}
A_{2}(I,t)= \frac{\sigma^{2}(t)}{\tau},
\end{equation}

with $\sigma^{2}(t) = \frac{1}{2}\tau h^{2}\lambda(t)$

To check our naive approach, we assume that the spikes arriving at the synapse can be modelled by in nonhomogeneous Poisson process. We use a sinusoidal rate function 

\begin{equation}
\lambda(t)=A\sin(\omega t)+\lambda_{0},
\end{equation}

with amplitude $A$, frequency $\omega$ and baseline firing rate $\lambda_{0}$. The frequency $\omega=2\pi/T$ can be expressed in terms of the period $T$. For $T \gg \tau$, the firing rate changes slowly compared the time constant of the synapse, so we assume that system has enough time to continuously adapt to the rate changes. For small $T$, we expect deviations.  


In [ ]:
def generate_nonhom_poisson_spike_train(lam, T, lam_max):
    '''Generate spike train from nonhomogeneous Poisson process with rate function r_t within recording time T
    :param lam: rate function
    :param T: recording time
    :param lam_max: max(lam) for t in [0,T]
    :return spike_times: array with spike times'''
    
    N = np.random.poisson(lam_max*T)
    
    # draw spike for homogeneous Poisson process with rate r_max
    hom_spike_times = np.sort(np.random.uniform(0, T, N)) 

    #thinning
    p = lam(hom_spike_times)/lam_max # probability to accept spike
    u = np.random.uniform(size = N)        
    
    spike_times = hom_spike_times[u <= p]
    
    return spike_times


def generate_nonhom_exp_decaying_current_trials(N, T, t, tau, h, lam, lam_max, I0):
    '''Generate N current traces for randomly drawn nonhomogeneous Poisson spike trains
    :param N: number of trials
    :param t: time points 
    :param tau: synaptic time constant
    :param h: synaptic efficacy
    :param lam: rate function
    :param lam_max: max(lam) for t in [0,T]
    :param I0: initial condition 
    :return I_mat: N x len(t) matrix with current traces
    '''
    
    I_mat = np.tile(np.exp(-t/tau), (N,1))

    if np.isscalar(I0): # fixed initial value
        I_mat = I0*I_mat         
    else: # initial values are drawn from distribution
        for i, I0_i in enumerate(I0):
            I_mat[i,:] *= I0_i
                       
    for i in xrange(N):
        
        spike_train = generate_nonhom_poisson_spike_train(lam, T, lam_max)
        I_mat[i, :] += exp_decaying_current(t, spike_train, tau, h)

    return I_mat


#------------------------------------------------------------------------------ 
# parameter

N = 100 # number of trials

tau = 2. # intrinsic synapse time constant
h = 0.1 # efficacy 
dt = 0.001*tau # time step

lam0 = 15. # baseline rate
A = 5. # sinusoidal amplitude

lam_max = A + lam0 # is needed to generate nonhomogeneous Poisson process from lam(t)

#------------------------------------------------------------------------------ 
# small period

T1 = tau # period
w1 = 2.*np.pi/T1 # frequency
Tr1 = 5*T1 # recording time 

t1 = np.arange(0, Tr1 + dt, dt) # time points 
M1 = len(t1) # number of time points

lam1 = lambda t: A*np.sin(w1*t) + lam0 # rate function

#------------------------------------------------------------------------------ 
# medium period

T2 = 5.*tau # period
w2 = 2.*np.pi/T2 # frequency
Tr2 = 5*T2 # recording time 

t2 = np.arange(0, Tr2 + dt, dt) # time points 
M2 = len(t2) # number of time points

lam2 = lambda t: A*np.sin(w2*t) + lam0 # rate function

#------------------------------------------------------------------------------ 
# large period

T3 = 20.*tau # period
w3 = 2.*np.pi/T3 # frequency
Tr3 = 5*T3 # recording time

t3 = np.arange(0, Tr3 + dt, dt) # time points 
M3 = len(t3) # number of time points

lam3 = lambda t: A*np.sin(w3*t) + lam0 # rate function

#------------------------------------------------------------------------------ 
# diffusion approximation

mu1 = tau*h*lam1(t1) # expected current at t
sig2_1 = 0.5*tau*h**2*lam1(t1) # expected variance at t

mu2 = tau*h*lam2(t2) # expected current at t
sig2_2 = 0.5*tau*h**2*lam2(t2) # expected variance at t

mu3 = tau*h*lam3(t3) # expected current at t
sig2_3 = 0.5*tau*h**2*lam3(t3) # expected variance at t

#------------------------------------------------------------------------------ 
# generate current trials

# draw initial values from stationary distribution at t=0
I0_1 = np.random.normal(mu1[0], np.sqrt(sig2_1[0]), N)
I0_2 = np.random.normal(mu2[0], np.sqrt(sig2_2[0]), N)
I0_3 = np.random.normal(mu3[0], np.sqrt(sig2_3[0]), N)

# generate current traces
I_mat1 = generate_nonhom_exp_decaying_current_trials(N, Tr1, t1, tau, h, lam1, lam_max, I0_1)
I_mat2 = generate_nonhom_exp_decaying_current_trials(N, Tr2, t2, tau, h, lam2, lam_max, I0_2)
I_mat3 = generate_nonhom_exp_decaying_current_trials(N, Tr3, t3, tau, h, lam3, lam_max, I0_2)

#------------------------------------------------------------------------------ 
# plotting

plt.figure(figsize = (14, 10))

gs = plt.GridSpec(3,1)

ax0 = plt.subplot(gs[0])

for I in I_mat1:
    
    ax0.plot(t1, I, '-', linewidth = 0.1)  

y_max = max(max(I_mat1.flatten()), max(I_mat1.flatten())) 

plt.plot(t1, mu1, color = 'k', linestyle = '-', label = r'$\mu$')
plt.plot(t1, mu1 - 2*np.sqrt(sig2_1), color = 'k', linestyle = '--', label = r'$\mu - 2\sigma$')
plt.plot(t1, mu1 + 2*np.sqrt(sig2_1), color = 'k', linestyle = '--', label = r'$\mu + 2\sigma$')

ax0.set_title(r'Small period $T = \tau$', fontsize = 18)    
plt.xlim(0, t1[-1])
plt.ylim(0, y_max)
plt.legend(loc = 'upper right', fontsize = 16)                      
plt.ylabel('$I(t)$', fontsize = 18)

ax1 = plt.subplot(gs[1])

for I in I_mat2:
    
    ax1.plot(t2, I, '-', linewidth = 0.1)  

y_max = max(max(I_mat2.flatten()), max(I_mat2.flatten())) 

plt.plot(t2, mu2, color = 'k', linestyle = '-', label = r'$\mu$')
plt.plot(t2, mu2 - 2*np.sqrt(sig2_2), color = 'k', linestyle = '--', label = r'$\mu - 2\sigma$')
plt.plot(t2, mu2 + 2*np.sqrt(sig2_2), color = 'k', linestyle = '--', label = r'$\mu + 2\sigma$')

ax1.set_title(r'Medium period $T > \tau$', fontsize = 18)    
plt.xlim(0, t2[-1])
plt.ylim(0, y_max)
plt.legend(loc = 'upper right', fontsize = 16)                      
plt.ylabel('$I(t)$', fontsize = 18)

ax2 = plt.subplot(gs[2])

for I in I_mat3:
    
    ax2.plot(t3, I, '-', linewidth = 0.1)  

y_max = max(max(I_mat3.flatten()), max(I_mat3.flatten())) 

plt.plot(t3, mu3, color = 'k', linestyle = '-', label = r'$\mu$')
plt.plot(t3, mu3 - 2*np.sqrt(sig2_3), color = 'k', linestyle = '--', label = r'$\mu - 2\sigma$')
plt.plot(t3, mu3 + 2*np.sqrt(sig2_3), color = 'k', linestyle = '--', label = r'$\mu + 2\sigma$')

ax2.set_title(r'Large period $T \gg \tau$', fontsize = 18)    
plt.xlim(0, t3[-1])
plt.ylim(0, y_max)
plt.legend(loc = 'upper right', fontsize = 16)                      
plt.ylabel('$I(t)$', fontsize = 18)

plt.show()

As expected, for fast changing input rates $\lambda(t)$, our naive approach does not predict the time evolution of the current traces adequately. We observe, that the current traces lag behind the expectation value $\mu(t)$ predicted from the stationary distribution. The variance is largest at rate maxima and lowest at the minima. 

In general, the one dimensional Langevin equation can be written as 

\begin{equation} \label{eq: Langevin}
\frac{dI}{dt}=h(I,t)+g(I,t)\xi(t),
\end{equation}

where $\xi(t)$ is a Gaussian white noise with zero mean and correlation function 


\begin{equation}
\bigl\langle\xi(t)\xi(t')\bigr\rangle=2\delta(t-t').
\end{equation}

Note that we absorbed the noise strength into the function $g(I,t)$. If $g(I,t)$ depends on the $I$, then we we call the noise term in \ref{eq: Langevin} multiplicative noise. If not, we call it additive. Hence, in our case we are dealing with a Langevin equation with additive noise. 

We want to choose $h(I,t)$ and $g(I,t)$ such that we recover the correct drift and diffusion coefficients. If so, then we could use \label{eq: Langevin} to predict time evolution of the first two moments of the system. In [1], the following relation is derived

\begin{align}
A_{1}(I,t) &= h(I,t)+g'(I,t)g(I,t) \\
A_{2}(I,t) &= g^{2}(I,t)
\end{align}

Hence, we identify 

\begin{align}
h(I,t) &= h(I,t) \\
g(t) &= \pm \frac{\sigma(t)}{\sqrt{\tau}},
\end{align}

which leads to the following Langevin equation 


\begin{equation} 
\frac{dI}{dt}=-\frac{1}{\tau}[I(t)-\mu(t)]+\frac{\sigma(t)}{\sqrt{\tau}}\xi(t).
\end{equation}

For the initial condition $I(0)=I_0$ the solution to the above equation is

\begin{equation}
I(t)=I_{0}e^{-\frac{t}{\tau}}+\frac{1}{\tau}e^{-\frac{t}{\tau}}\int_{0}^{t}dt'\,\left[\mu(t')+\sqrt{\tau}\sigma(t')\xi(t')\right]e^{\frac{t'}{\tau}}
\end{equation}

Taking the expectation value on both sides and using $\left\langle \xi(t)\right\rangle =0$ yields 

\begin{equation}
\left\langle I(t)\right\rangle =I_{0}e^{-\frac{t}{\tau}}+\frac{1}{\tau}e^{-\frac{t}{\tau}}\int_{0}^{t}dt'\mu(t')e^{\frac{t'}{\tau}}
\end{equation}

For a sinusoidal rate function $\lambda(t)=A\sin(\omega t)+\lambda_{0}$ we obtain 

\begin{equation}
I_{0}e^{-\frac{t}{\tau}}+\left(\tau h\lambda_{0}-e^{-\frac{t}{\tau}}\right)+A\frac{\tau^{2}\omega}{1+\omega^{2}\tau^{2}}e^{-\frac{t}{\tau}}+\tau hA\frac{\sin(\omega t)-\tau\omega\cos(\omega t)}{1+\omega^2\tau^{2}}
\end{equation}

The exponential equations in the above equation tell us that the initial condition do not matter for $t\gg \tau$. After some initial transients, the equation simplifies to 

\begin{equation}
\left\langle I(t)\right\rangle =\tau h\lambda_{0}+\tau hA\frac{\sin(\omega t)-\tau\omega\cos(\omega t)}{1+\omega^2\tau^{2}}
\end{equation}

If the rate changes on much slower time scale $T>>\tau$ then the above equation simplifies to     

\begin{equation} \label{eq: exp I}
\left\langle I(t)\right\rangle =\tau h\lambda_{0}+\tau hA\sin(\omega t)=\tau h\lambda(t),
\end{equation}

where we used  $\omega = 2 \pi /T$. Note that this is in agreement with our observation in panel three. Equation \ref{} can  be written as 

\begin{equation}
\left\langle I(t)\right\rangle = \mu_0 + \delta \mu(t),
\end{equation}

where we defined 

\begin{align}
\mu_0 &= \tau h\lambda_{0} \\
\delta \mu(t) &= \tau hA\frac{\sin(\omega t)-\tau\omega\cos(\omega t)}{1+\omega^2\tau^{2}}
\end{align}

Next, we want to calculate the correlation function $a(t_1, t_2) = \left\langle I(t_1)I(t_2)\right\rangle$. To simplfy the derivation we set $I_0 = 0$. This will not change the result for $t\gg \tau$ which is independent of the initial conditions as we already saw. Hence, we need to solve    

\begin{equation} 
a(t_1, t_2) =\left\langle \frac{1}{\tau^{2}}e^{-\frac{t_{1}+t_{2}}{\tau}}\int_{0}^{t_{1}}dt'_{1}\int_{0}^{t_{2}}dt'_{2}\,\left[\mu(t'_{1})+\sqrt{\tau}\sigma(t'_{1})\xi(t'_{1})\right]\left[\mu(t'_{2})+\sqrt{\tau}\sigma(t'_{2})\xi(t'_{2})\right]e^{\frac{t'_{1}}{\tau}}e^{\frac{t'_{2}}{\tau}}\right\rangle 
\end{equation}

Using $\left\langle \xi(t)\right\rangle =0$ the integral can separated into two contributions

\begin{equation} \label{eq: a}
a(t_1, t_2) =\frac{1}{\tau^{2}}e^{-\frac{t_{1}+t_{2}}{\tau}}\int_{0}^{t_{1}}dt'_{1}\,\mu(t'_{1})e^{\frac{t'_{1}}{\tau}}\int_{0}^{t_{2}}dt'_{2}\,\mu(t'_{2})e^{\frac{t'_{2}}{\tau}}+\frac{1}{\tau}e^{-\frac{t_{1}+t_{2}}{\tau}}\int_{0}^{t_{1}}dt'_{1}\int_{0}^{t_{2}}dt'_{2}\,\sigma(t'_{1})\sigma(t'_{2})\left\langle \xi(t'_{1})\xi(t'_{2})\right\rangle e^{\frac{t'_{1}}{\tau}}e^{\frac{t'_{2}}{\tau}}
\end{equation}

Comparintg to \ref{eq: exp I}, we notice that the first term is equivalent to the product of $\left\langle I(t_1)\right\rangle$ and $\left\langle I(t_2)\right\rangle$. Hence, the centralized correaltion function $\overline{a}(t_{1},t_{2})=\left\langle I(t_{1})I(t_{2})\right\rangle -\left\langle I(t_{1})\right\rangle \left\langle I(t_{2})\right\rangle$ is given by the second term in \ref{eq: a} 

\begin{equation}
\overline{a}(t_{1},t_{2})=\frac{1}{\tau}e^{-\frac{t_{1}+t_{2}}{\tau}}\int_{0}^{t_{1}}dt'_{1}\int_{0}^{t_{2}}dt'_{2}\,\sigma(t'_{1})\sigma(t'_{2})\left\langle \xi(t'_{1})\xi(t'_{2})\right\rangle e^{\frac{t'_{1}}{\tau}}e^{\frac{t'_{2}}{\tau}}
\end{equation}

Using $\left\langle \xi(t'_{1})\xi(t'_{2})\right\rangle =2\delta(t'_{1}-t'_{2})$, we get rid of one of the integrals leading to 

\begin{equation}
\overline{a}(t_{1},t_{2})=\frac{2}{\tau}e^{-\frac{t_{1}+t_{2}}{\tau}}\int_{0}^{\text{min}(t_{1},t_{2})}dt'_{1}\,\sigma^{2}(t'_{1})e^{2\frac{t'_{1}}{\tau}}
\end{equation}

Inserting $\sigma^2(t)=\tau h^2 \lambda(t)$ and $\lambda(t)=\lambda(t)=A\sin(\omega t)+\lambda_{0}$ yields and assuming $t_1 > t_2$, we need to solve  

\begin{equation}
\overline{a}(t_{1},t_{2})=h^{2}e^{-\frac{t_{1}+t_{2}}{\tau}}\int_{0}^{t_{2}}dt'_{1}\,\left(A\sin(\omega t'_{1})+\lambda_{0}\right)e^{2\frac{t'_{1}}{\tau}}
\end{equation}

which yields 

\begin{equation}
\overline{a}(t_{1},t_{2}) = \frac{h^{2}\tau\lambda_{0}}{2}\left(e^{-\frac{t_{1}-t_{2}}{\tau}}-e^{-\frac{t_{1}+t_{2}}{\tau}}\right)+Ah^{2}\tau\left(\frac{\tau\omega}{\tau^{2}\omega^{2}+4}e^{-\frac{t_{1}+t_{2}}{\tau}}+\frac{e^{-\frac{t_{1}+t_{2}}{\tau}}\left(2\sin(\omega t_{2})-\tau\omega\cos(\omega t_{2})\right)}{\tau^{2}\omega^{2}+4}\right)
\end{equation}

For $t_1,t_2 \gg \tau$, the above equation simplifies to 

\begin{equation}
\overline{a}_{\infty}(t_{1},t_{2})=\frac{h^{2}\tau\lambda_{0}}{2}e^{-\frac{\left|t_{1}-t_{2}\right|}{\tau}}+Ah^{2}\tau\frac{e^{-\frac{\left|t_{1}-t_{2}\right|}{\tau}}\left(2\sin(\omega \text{min}[t_{1},t_{2}])-\tau\omega\cos(\omega \text{min}[t_{1},t_{2}])\right)}{\tau^{2}\omega^{2}+4}
\end{equation}

We notice the following. The centralized correlation function is damped by the time different $\left|t_{1}+t_{2}\right|$ but also depends on the time point $t_{2}$ of observation because the rate is not stationary anymore. Hence, we arrive at the following expression for the zero lag correlation

\begin{equation} \label{eq: zero lag a}
\overline{a}_{\infty}(t)= \frac{h^{2}\tau\lambda_{0}}{2} + Ah^{2}\tau\frac{\left(2\sin(\omega t)-\tau\omega\cos(\omega t)\right)}{\tau^{2}\omega^{2}+4}
\end{equation}

which can be written as 

\begin{equation} 
\overline{a}_{\infty}(t)= \sigma_{0}^{2} + \delta\sigma^{2}(t)
\end{equation}


where we defined 

\begin{align}
\sigma_{0}^{2}	&= \frac{h^{2}\tau\lambda_{0}}{2} \\
\delta\sigma^{2}(t)	&= Ah^{2}\tau\frac{\left(2\sin(\omega t)-\tau\omega\cos(\omega t)\right)}{\tau^{2}\omega^{2}+4}
\end{align}

Notice, that if  $T>>\tau$ then we recover of naive expectation. We check now if \ref{eq: exp I} and \ref{eq: zero lag a} provide a better prediction for the time evolution of and the fluctuations of $I$ in case that the rate changes on a time scale similar to the intrinsic synaptic time scale.   

In [ ]:
#------------------------------------------------------------------------------ 
# improved diffusion approximation
mu0 = tau*h*lam0
sig2_0 = h**2*tau*lam0/2

mu1 = mu0 + tau*h*A*(np.sin(w1*t1) - tau*w1*np.cos(w1*t1))/(1+(w1*tau)**2) # expected current at t
sig2_1 = sig2_0 + A*h**2*tau*(2*np.sin(w1*t1) - tau*w1*np.cos(w1*t1))/(4+(w1*tau)**2) # expected variance at t

mu2 = mu0 + tau*h*A*(np.sin(w2*t2) - tau*w2*np.cos(w2*t2))/(1+(w2*tau)**2)
sig2_2 = sig2_0 + A*h**2*tau*(2*np.sin(w2*t2) - tau*w2*np.cos(w2*t2))/(4+(w2*tau)**2)# expected variance at t

mu3 = mu0 + tau*h*A*(np.sin(w3*t3) - tau*w3*np.cos(w3*t3))/(1+(w3*tau)**2)
sig2_3 = sig2_0 + A*h**2*tau*(2*np.sin(w3*t3) - tau*w3*np.cos(w3*t3))/(4+(w3*tau)**2)# expected variance at t


#------------------------------------------------------------------------------ 
# plotting

plt.figure(figsize = (14, 10))

gs = plt.GridSpec(3,1)

ax0 = plt.subplot(gs[0])

for I in I_mat1:
    
    ax0.plot(t1, I, '-', linewidth = 0.1)  

y_max = max(max(I_mat1.flatten()), max(I_mat1.flatten())) 

plt.plot(t1, mu1, color = 'k', linestyle = '-', label = r'$\mu$')
plt.plot(t1, mu1 - 2*np.sqrt(sig2_1), color = 'k', linestyle = '--', label = r'$\mu - 2\sigma$')
plt.plot(t1, mu1 + 2*np.sqrt(sig2_1), color = 'k', linestyle = '--', label = r'$\mu + 2\sigma$')

ax0.set_title(r'Small period $T = \tau$', fontsize = 18)    
plt.xlim(0, t1[-1])
plt.ylim(0, y_max)
plt.legend(loc = 'upper right', fontsize = 16)                      
plt.ylabel('$I(t)$', fontsize = 18)

ax1 = plt.subplot(gs[1])

for I in I_mat2:
    
    ax1.plot(t2, I, '-', linewidth = 0.1)  

y_max = max(max(I_mat2.flatten()), max(I_mat2.flatten())) 

plt.plot(t2, mu2, color = 'k', linestyle = '-', label = r'$\mu$')
plt.plot(t2, mu2 - 2*np.sqrt(sig2_2), color = 'k', linestyle = '--', label = r'$\mu - 2\sigma$')
plt.plot(t2, mu2 + 2*np.sqrt(sig2_2), color = 'k', linestyle = '--', label = r'$\mu + 2\sigma$')

ax1.set_title(r'Medium period $T > \tau$', fontsize = 18)    
plt.xlim(0, t2[-1])
plt.ylim(0, y_max)
plt.legend(loc = 'upper right', fontsize = 16)                      
plt.ylabel('$I(t)$', fontsize = 18)

ax2 = plt.subplot(gs[2])

for I in I_mat3:
    
    ax2.plot(t3, I, '-', linewidth = 0.1)  

y_max = max(max(I_mat3.flatten()), max(I_mat3.flatten())) 

plt.plot(t3, mu3, color = 'k', linestyle = '-', label = r'$\mu$')
plt.plot(t3, mu3 - 2*np.sqrt(sig2_3), color = 'k', linestyle = '--', label = r'$\mu - 2\sigma$')
plt.plot(t3, mu3 + 2*np.sqrt(sig2_3), color = 'k', linestyle = '--', label = r'$\mu + 2\sigma$')

ax2.set_title(r'Large period $T \gg \tau$', fontsize = 18)    
plt.xlim(0, t3[-1])
plt.ylim(0, y_max)
plt.legend(loc = 'upper right', fontsize = 16)                      
plt.ylabel('$I(t)$', fontsize = 18)

plt.show()

## Superposition of multiple synapses

Here, we want to look at the superposition of multiple currents from different synapses each having possibly different time constants and or efficacies. For the general case of $N$ different synapses, the total input current is given 

\begin{equation} \label{eq: I}
I(t)=\sum_{i=1}^{N}I_{i}(t)
\end{equation}

The dynamics for each current $I_i$ are described by 

\begin{equation} \label{eq: deq I_i}
\frac{dI_i(t)}{dt}=-\frac{1}{\tau_{i}} I_i(t)+h_i s_i(t)
\end{equation}

with synaptic time constant $\tau_i$ and efficacy $h_i$. In zeros order approximation, we assume that all spike trains $s_i(t)$ can be modeled by independent Poisson processes with possibly different rates $\lambda_i$. However, note that it is very likely that individual neurons connect two multiple synapses which would introduce correlations between the different spike trains $s_i(t)$. 

In the diffusion approximation, $I_i$ approaches a Gaussian distributed $\rho_i(I)$ in the stationary limit with mean  $\mu_i = \tau_i \lambda_i h_i$ and variance $\sigma^{2}=\tau_i \lambda_i h_i^2$/2. As we already discussed, we can effectively replace \ref{eq: deq I_i} by a Langevin equation with Gaussian input noise. 

\begin{equation} 
\frac{dI_i(t)}{dt}=-\frac{1}{\tau_i} (I_i(t) - \mu_i) +\xi_i(t) 
\end{equation},

The noise has zero mean $\left\langle \xi_i(t)\right\rangle =0 $ and delta shaped autocorrelation function   

\begin{equation}
\quad\left\langle \xi_i(t)\xi_i(t')\right\rangle =\tilde{\sigma}_i^{2}\delta(t-t')
\end{equation}.

We showed that the noise strength $\tilde{\sigma}_i^{2}$ must be set to $\tilde{\sigma_i}^2=2\sigma_i^2/\tau_i$ to arrive at the same diffusion coefficient obtained during the derivation of the Fokker-Planck equation.

Going on, our goal is calculate the stationary distribution for the total input current \ref{eq: I}. In the stationary limit, $I$ is simply the sum of Gaussian distributed random variables. Thus, we expect $I$ to be Gaussian distributed with mean $\mu$ and variance $\sigma^2$. The mean $\mu$ can be obtained from the stationary limit of the expected time evolution of $I(t)$

\begin{equation} 
\left\langle I(t)\right\rangle =\sum_{i=1}^{N} \left\langle I_i(t)\right\rangle 
\end{equation}

We calculated the expected time evolution $\left\langle I_{i}(t)\right\rangle$ for an individual synapse already in the previous section. Hence, for initial conditions $I_i(0)=0$, we find 

\begin{equation} 
\left\langle I(t)\right\rangle =\sum_{i=1}^{N} \mu_i \left(1-e^{-\frac{t}{\tau_i}}\right) 
\end{equation}

In the stationary limit with $t \gg max(\tau_i)$, the above equation reduces to   

\begin{equation} \label{eq: mu}
\mu = \left\langle I(\infty)\right\rangle =\sum_{i=1}^{N}\mu_{i}
\end{equation}.

The variance $\sigma^2$ of the stationary distribution can be obtained from the centralized zero lag autocorrelation function. The autocorrelation $a(t_1, t_2) = \left\langle I(t_{1})I(t_{2})\right\rangle$function is given by  

\begin{equation}
a(t_1, t_2) =\sum_{i,j=1}^{N}\left\langle I_{i}(t_{1})I_{j}(t_{2})\right\rangle 
\end{equation}

For initial conditions $I_i = 0$, using that the synaptic currents are independent, we find 

\begin{equation} 
a(t_1, t_2) =\sum_{i,j=1}^{N}\left[\mu_{i}\mu_{j}(1-e^{-\frac{t}{\tau_{\alpha}}})(1-e^{-\frac{t}{\tau_{\beta}}})+\delta_{ij}\frac{\tau_{i}\tilde{\sigma}_{i}^{2}}{2}\left(e^{-\frac{\left|t_{1}-t_{2}\right|}{\tau_{i}}}+e^{-\frac{t_{1}+t_{2}}{\tau_{i}}}\right)\right],
\end{equation}

In the stationary limit with $t \gg max(\tau_i)$, the above equation reduces to

\begin{equation} \label{eq: I autocorrelation}
a_{\infty}(\left|t_{1}-t_{2}\right|)  =\sum_{i,j=1}^{N}\mu_{i}\mu_{j}+\sum_{i=1}^{N}\frac{\tau_{i}\tilde{\sigma}_{i}^{2}}{2}e^{-\frac{\left|t_{1}-t_{2}\right|}{\tau_{i}}}
\end{equation},

which depends only on the time difference $\left|t_{1}-t_{2}\right|$. Thus, the zero lag autocorrelation function $a_{\infty}(0)$ is given by

\begin{equation}
a_{\infty}(0) = \mu^{2}+\sum_{i=1}^{N}\frac{\tau_{i}\tilde{\sigma}_{i}^{2}}{2}
\end{equation},

where we used that first sum \ref{eq: I autocorrelation} is equivalent to $\mu^2$ defined in \ref{eq: mu}. Inserting $\tilde{\sigma_i}^2=2\sigma_i^2/\tau_i$, we arrive at the following expression for the centralized zero lag correlation function 

\begin{equation} \label{I variance}
\sigma^2 = a_{\infty}(0)-\mu^{2}=\sum_{i=1}^{N}\sigma_{i}^{2}
\end{equation}.

Hence, we find that the mean \ref{eq: mu} and the variance \ref{I variance} of the stationary distribution for the total current \ref{eq: I} is simply given by the summed means and summed variances of the stationary distributions of all individual currents as one would expect for the sum of Gaussian distributed random variables.   

We now consider the case of one excitatory and one inhibitory synapse, i.e. $I(t)$ is given 

\begin{equation} \label{eq: deq I}
I(t)= I_E(t) + I_I(t),
\end{equation}

Thus, under the assumption that both synapses receive independent Poisson spike trains, the stationary distribution $\rho(I)$ is simply a Gaussian distribution 

\begin{equation} \label{eq: stationary rho EI}
\rho(I) = \frac{1}{\sqrt{2\pi}\sigma}e^{-\frac{(I-\mu)^{2}}{2 \sigma^{2}}}.
\end{equation},

with mean $\mu=\mu_E + \mu_I$ and variance $\sigma^2 = \sigma_E^2 + \sigma_I^2$ and $\mu_{\alpha} = \tau_{\alpha} \lambda_{\alpha} h_{\alpha}$ and $\sigma^2_{\alpha} = \tau_{\alpha} \lambda_{\alpha} h_{\alpha}^2$. 

To check \ref{eq: stationary rho EI}, we will generate an ensemble of trials from which we calculate the stationary distribution. 

In [ ]:
#------------------------------------------------------------------------------ 
# parameter

# excitatory synapse
lamE = 50. # rate
tauE = 1 # intrinsic synapse time constant
hE = 0.1 # efficacy 

# inhibitory synapse
lamI = 5. # rate
tauI = 2 # intrinsic synapse time constant
hI = -0.25 # efficacy 

N = 100 # number of trials
T = 20 # recording time 

dt = 0.001*min(tauE, tauI) # time step
t = np.arange(0, T + dt, dt) # time array
M = len(t) # numbe of time steps

#------------------------------------------------------------------------------ 
# diffusion approximation

muE = tauE*lamE*hE 
sigE2 = 0.5*tauE*lamE*hE**2

muI = tauI*lamI*hI 
sigI2 = 0.5*tauI*lamI*hI**2

mu = muE + muI
sig2 = sigE2 + sigI2

#------------------------------------------------------------------------------ 
# generate current trials

# initial condition drawn from stationary distribution     
I0_E = np.random.normal(muE, np.sqrt(sigE2), N)
I0_I = np.random.normal(muI, np.sqrt(sigI2), N)

I_mat_E = generate_exp_decaying_current_trials(N, T, t, tauE, hE, lamE, I0_E)
I_mat_I = generate_exp_decaying_current_trials(N, T, t, tauI, hI, lamI, I0_I)

I_mat = I_mat_E + I_mat_I
    
plt.figure(figsize = (14, 10))

gs = plt.GridSpec(1,1)

ax0 = plt.subplot(gs[0])

for I in I_mat:
    
    ax0.plot(t, I, '-', linewidth = 0.1)  

y_max = max(I_mat.flatten()) 
y_min = min(I_mat.flatten())

plt.hlines(mu, 0, t[-1], colors = 'k', linestyles = '-', label = r'$\mu$')
plt.hlines(mu - 2*np.sqrt(sig2), 0, t[-1], colors = 'k', linestyles = '--', label = r'$\mu - 2\sigma$')
plt.hlines(mu + 2*np.sqrt(sig2), 0, t[-1], colors = 'k', linestyles = '--', label = r'$\mu + 2\sigma$')

ax0.set_title(r'Initial condition drawn from stationary distribution', fontsize = 18)    
plt.xlim(0, t[-1])
plt.ylim(y_min, y_max)
plt.legend(loc = 'upper right', fontsize = 16)

plt.show()

The initial transient seems to be caused by the different time constants $\tau_E$ and $\tau_I$. To obtain the stationary distribution, we need to collect data after some relaxation period $T_r \gg \max(\tau_E, \tau_I)$. 

In [ ]:
#------------------------------------------------------------------------------ 
# Stationary distribution

Tr= 5*max(tauE, tauI) 

I_mat = I_mat[:, t >= Tr]

I_arr = I_mat.flatten() # flatten time resolved current trials

plt.figure(figsize = (14, 6))

plt.hist(I_arr, 100, density = True)

plt.xlabel(r'$I$', fontsize = 18)
plt.ylabel(r'$\rho(I)$', fontsize = 18)

# compare to stationary distribution obtained from Fokker-Planck equation

I = np.linspace(min(I_arr), max(I_arr), 1000) 
rho = 1./np.sqrt(2*np.pi*sig2)*np.exp(-0.5*(I-mu)**2/sig2)

plt.plot(I, rho, '-', color = 'red', linewidth = 2, label = r'$\rho(I)$ stationary')
plt.legend(loc = 'upper right', fontsize = 16)
plt.show()


## Membrane potential

Our goal is to derive the stationary distribution of membrane potential for Leaky-Integrate and Fire neuron which receives a input current from  multiple exponential current-based synapses which themselves are driven by independent Poisson spike trains. The dynamics of the membrane potential are described by  

\begin{equation} \label{eq: deq V}
\frac{dV(t)}{dt}=-\frac{1}{\tau_{m}} V + I(t)
\end{equation},

with 

\begin{equation} \label{eq: I}
I(t)=\sum_{i=1}^{N}I_{i}(t)
\end{equation}

Independent of the details of $I(t)$, the formal solution to \ref{eq: deq V} for the initial condition $V(0)=V_0$ is 

\begin{equation} 
V(t)=V_{0}e^{-\frac{t}{\tau_{m}}}+\int_{0}^{t}dt'\,e^{-\frac{t-t'}{\tau}}I(t').
\end{equation}

To obtain the mean $mu_V$ of the stationary can be obtained from the stationary limit of the expected time evolution $\left\langle V(t)\right\rangle$. Note that $I(t)$ is the only stochastic quantity in the above equation. Hence, setting $V_0 = 0$, $\left\langle V(t)\right\rangle$ is given by 

\begin{equation}
a(t_1, t_2) = \left\langle V(t)\right\rangle =\int_{0}^{t}dt'\,e^{-\frac{t-t'}{\tau}}\left\langle I(t')\right\rangle.
\end{equation}

Inserting our result for $\left\langle I(t)\right\rangle$ from the previous section and solving the integral yields 

\begin{equation}
\left\langle V(t)\right\rangle =\tau_{m}\sum_{i=1}^{N}\mu_{i}\left[1-e^{-\frac{t}{\tau_{i}}}+\frac{\tau_{i}}{\tau_{m}-\tau_{i}}\left(e^{-\frac{t}{\tau_{i}}}-e^{-\frac{t}{\tau_{m}}}\right)\right].
\end{equation}

Thus, in the stationary limit $t/\tau_m \gg 1$, we obtain 

\begin{equation} \label{eq: V mean}
\mu_{V}=V(\infty)=\tau_m \mu
\end{equation}

with $\mu=\sum_{i}\mu_{i}$. The variance $\sigma_V^2$ of the stationary distribution can be obtained from the centralized zero lag autocorrelation function The autocorrelation function $\left\langle V(t_{1})V(t_{2})\right\rangle$ is given by   

\begin{equation}
a(t_1, t_2) = \bigl\langle V(t_{1})V(t_{2})\bigr\rangle=\int_{0}^{t_{1}}dt'_{1}\int_{0}^{t_{2}}dt'_{2}\,e^{-\frac{t_{1}+t_{2}-t'_{1}-t_{2}'}{\tau_{m}}}\bigl\langle I(t'_{1})I(t'_{2})\bigr\rangle
\end{equation}

Inserting our results for autocorrelation function $\bigl\langle I(t'_{1})I(t'_{2})\bigr\rangle$ from previous sections and solving the integrals results in quite complicated expression. However, in the stationary limit $t/\tau_m \gg 1$, the above equation reduces to

\begin{equation}
a_{\infty}(0)=\tau_{m}\sum_{i,j=1}^{N}\mu_{i}\mu_{j}+\sum_{i=1}^{N}\frac{\tau_{i}\tilde{\sigma}_{i}^{2}}{2}\left[\frac{\tau_{m}^{2}\tau_{i}}{\tau_{m}+\tau_{i}}e^{-\frac{\left|t_{1}-t_{2}\right|}{\tau_{m}}}+\frac{\tau_{m}^{2}\tau_{i}^{2}}{\tau_{i}^{2}-\tau_{m}^{2}}\left(e^{-\frac{\bigl|t_{1-}t_{2}\bigr|}{\tau_{i}}}-e^{-\frac{\bigl|t_{1}-t_{2}\bigr|}{\tau_{m}}}\right)\right]
\end{equation}

which depends only on the time difference $\left|t_{1}-t_{2}\right|$ as expected. Hence, for the zero lag autocorrelation function $a_{\infty}$(0)$, we find

\begin{equation}
a_{\infty}(\left|t_{1}-t_{2}\right|)=\tau_{m}\sum_{i,j=1}^{N}\mu_{i}\mu_{j}+\tau_{m}^{2}\sum_{i=1}^{N}\frac{\tilde{\sigma}_{i}^{2}}{2}\frac{\tau_{i}^{2}}{\tau_{m}+\tau_{i}}
\end{equation}

Inserting $\tilde{\sigma}_{\alpha}^{2}=2\sigma_{\alpha}^{2}/\tau_{\alpha}$, we arrive at the following expression for the centralized zero lag correlation function 

\begin{equation} \label{eq: zero lag stationary correlation function}
\sigma_{V}^{2}=a_{\infty}(0)-\mu_{V}^{2}=\tau_{m}^{2}\sum_{i=1}^{N}\frac{\tau_{i}}{\tau_{m}+\tau_{i}}\sigma_{i}^{2}
\end{equation}

Thus, we expect the stationary distribution of membrane potential to be given by

\begin{equation}
\rho(V)=\frac{1}{\sqrt{2\pi}\sigma_{V}}e^{-\frac{(V-\mu_V)^{2}}{2\sigma_{V}^{2}}},
\end{equation}

with mean $mu_V$ given by \ref{eq: V mean} and $\sigma_{V}^{2}$ given by \ref{eq: zero lag stationary correlation function}. In case of one excitatory and one inhibitory synapse, $m_V$ is given by and $m_V = \tau_m(\mu_E + mu_I)$ with $\mu_{\alpha} = \tau_{\alpha} \lambda_{\alpha} h_{\alpha}$ and $\sigma_V^2$ is given by 

\begin{equation}
\sigma_{V}^{2}=\tau_{m}^{2}\left(\frac{\tau_{E}}{\tau_{m}+\tau_{E}}\sigma_{E}^{2}+\frac{\tau_{I}}{\tau_{m}+\tau_{I}}\sigma_{I}^{2}\right)
\end{equation}

For $\tau_m gg max(\tau_E, \tau_I)$, the above equation reduces to 

\begin{equation}
\sigma_{V}^{2}=\tau_{m}\left(\tau_{E}\sigma_{E}^{2}+\tau_{I}\sigma_{I}^{2}\right)
\end{equation}

To check this, we will generate an ensemble of trials with different realizations of the Poisson spike trains driving the input currents. 

In [ ]:
def generate_voltage_trials(N, t, tau, V0, I_mat):
    '''Generate N voltage traces for N different current inputs unsing euler method
    :param N: number of trials
    :param t: time points 
    :param tau: membrane time constant
    :param V0: initial value
    :param I_mat: N x len(t) matrix with current traces
    :return V_mat: N x len(t) matrix with voltage traces
    '''
    M = len(t) # number of time steps
    dt = t[1] - t[0] # step width 
        
    V_mat = np.tile(np.exp(-t/tau), (N,1))

    # initial value
    if np.isscalar(V0):
        V_mat = V0*V_mat         
    else:
        for i, V0_i in enumerate(V0):
            V_mat[i,:] *= V0_i  

    dt = t[1] - t[0]

    kernel = np.ones((M, M))
    kernel = np.triu(kernel, 0) # columns are heaviside functions for sliding t
             
    for i, ti in  enumerate(t):         
        kernel[:, i] =  kernel [:, i]*np.exp(-(ti-t)/tau)

    V_mat = np.matmul(I_mat, kernel)*dt
                                                                                  
    return V_mat  

#------------------------------------------------------------------------------ 
# parameter

# excitatory synapse
lamE = 50. # rate
tauE = 1 # intrinsic synapse time constant
hE = 0.1 # efficacy 

# inhibitory synapse
lamI = 5. # rate
tauI = 2 # intrinsic synapse time constant
hI = -0.25 # efficacy 

tau_m = 10 # membrane time constant

N = 500 # number of trials
T = 100 # recording time 

dt = 0.01*min((tauE, tauI, tau_m)) # time step
t = np.arange(0, T + dt, dt) # time points 
M = len(t) # number of time points

#------------------------------------------------------------------------------ 
# diffusion approximation

# current
muE = tauE*lamE*hE 
sigE2 = 0.5*tauE*lamE*hE**2

muI = tauI*lamI*hI 
sigI2 = 0.5*tauI*lamI*hI**2

mu = muE + muI # expectation value
sig2 = sigE2 + sigI2 # variance

# voltage
muV = tau_m*mu # expectation value  
sig2_V = tau_m**2*(tauE/(tauE + tau_m)*sigE2 + tauI/(tauI + tau_m)*sigI2) # variance

#------------------------------------------------------------------------------ 
# generate current trials

# initial condition drawn from stationary distribution     
I0_E = np.random.normal(muE, np.sqrt(sigE2), N)
I0_I = np.random.normal(muI, np.sqrt(sigI2), N)

# sharp initial condition      
I0_E = 0
I0_I = 0.

I_mat_E = generate_exp_decaying_current_trials(N, T, t, tauE, hE, lamE, I0_E)
I_mat_I = generate_exp_decaying_current_trials(N, T, t, tauI, hI, lamI, I0_I)

I_mat = I_mat_E + I_mat_I
        
#------------------------------------------------------------------------------ 
# generate voltage trials

V0 = 0. # sharpe initial condition 

V_mat = generate_voltage_trials(N, t, tau_m, V0, I_mat)

#------------------------------------------------------------------------------ 
# plotting

plt.figure(figsize = (14, 10))
 
gs = plt.GridSpec(2,1)
 
ax0 = plt.subplot(gs[0])
 
for I in I_mat:
     
    ax0.plot(t, I, '-', linewidth = 0.1)  
 
y_max = max(I_mat.flatten()) 
y_min = min(I_mat.flatten())
 
plt.hlines(mu, 0, t[-1], colors = 'k', linestyles = '-', label = r'$\mu$')
plt.hlines(mu - 2*np.sqrt(sig2), 0, t[-1], colors = 'k', linestyles = '--', label = r'$\mu - 2\sigma$')
plt.hlines(mu + 2*np.sqrt(sig2), 0, t[-1], colors = 'k', linestyles = '--', label = r'$\mu + 2\sigma$')
 
ax0.set_title(r'Sharp initial condition I(O)=I', fontsize = 18)    
plt.xlim(0, t[-1])
plt.ylim(y_min, y_max)
plt.legend(loc = 'upper right', fontsize = 16)    
 
ax1 = plt.subplot(gs[1])
 
for V in V_mat:
     
    ax1.plot(t, V, '-', linewidth = 0.5)  
 
y_max = max(V_mat.flatten()) 
y_min = min(V_mat.flatten())
 
plt.hlines(muV, 0, t[-1], colors = 'k', linestyles = '-', label = r'$\mu$')
plt.hlines(muV - 2*np.sqrt(sig2_V), 0, t[-1], colors = 'k', linestyles = '--', label = r'$\mu - 2\sigma$')
plt.hlines(muV + 2*np.sqrt(sig2_V), 0, t[-1], colors = 'k', linestyles = '--', label = r'$\mu + 2\sigma$')
 
ax1.set_title(r'Sharp initial condition V(0)=V', fontsize = 18)    
plt.xlim(0, t[-1])
plt.ylim(y_min, y_max)
plt.legend(loc = 'upper right', fontsize = 16)    
     
plt.show() 

To estimate the stationary distribution, we make a histogram of all different voltage traces. Since we are interested in the stationary distribution, we only collect data after some initial relaxation period $T_r \gg max(\tau_m)$ so that the above transient does not contribute.   

In [ ]:
#------------------------------------------------------------------------------ 
# Stationary distribution

Tr= 5*max((tauE, tauI, tau_m)) 

V_mat = V_mat[:, t >= Tr]

V_arr = V_mat.flatten() # flatten time resolved current trials

plt.figure(figsize = (14, 6))

plt.hist(V_arr, 100, density = True)

plt.xlabel(r'$I$', fontsize = 18)
plt.ylabel(r'$\rho(I)$', fontsize = 18)

# compare to stationary distribution obtained from Fokker-Planck equation

V = np.linspace(min(V_arr), max(V_arr), 1000) 
rho = 1./np.sqrt(2*np.pi*sig2_V)*np.exp(-0.5*(V-mu_V)**2/sig2_V)

plt.plot(V, rho, '-', color = 'red', linewidth = 2, label = r'$\rho(I)$ stationary')
plt.legend(loc = 'upper right', fontsize = 16)
plt.show()

Is it possible to arrive at the same distribution for the membrane potential for different parameter sets. In the case of two synapses, we have six parameter $\{\tau_1, h_1, \lambda_1, \tau_2, h_2, \lambda_2\}$ which determine the statistics of the two input currents plus the time constant $\tau_m$ of the membrane potential. On the other hand, we only have two parameter $\mu_V$ and $\sigma^2_V$ which are both functions of all model parameters, and which fully determine the stationary distribution of the membrane potential in second order approximation.
Thus, for a given $\mu_V$ and $\sigma^2_V$, we will not be able to determine all parameter uniquely. 

## Two dimensional Fokker-Planck

Here, we want to analyse the time evolution of the membrane potential using the multi-dimensional Fokker-Planck equation. For a single synapse, we have the following system of stochastic coupled differential equations 

\begin{equation} 
\frac{dV(t)}{dt}=-\frac{1}{\tau_{m}} V + I(t)
\end{equation},

and 

\begin{equation} 
\frac{dI(t)}{dt}=-\frac{1}{\tau}I(t)+ h s(t),
\end{equation}.

Replace spike train by Gaussian random variable $\xi(t)$ with mean $\mu_I$ and variance $\sigma^2_I$. Drift coefficient are $D_I$ and $D_V$ defined as 

\begin{equation}
D_i(x_i,t)=\lim_{\tau\rightarrow0}\frac{1}{\tau}\left.\bigl\langle[X_i(t+\tau)-x\bigr\rangle\right|_{t=0}
\end{equation}







In [ ]:
bins = np.linspace(0, mu+3*np.sqrt(sig2), 100)
hist = lambda arr, bins: np.histogram(arr, bins, density = True)[0]
rho_I = np.apply_along_axis(hist, 0, I_mat, bins)

bin_width = bins[1] - bins[0] 
I_bin = bins[:-1] +  bin_width


y_max = 1

fig = plt.figure(figsize = (14, 6))
ax = fig.add_subplot(111, autoscale_on=False, xlim=(0, mu+3*np.sqrt(sig2)), ylim=(0, y_max))

ax.plot([mu, mu], [0, y_max], '-k')
ax.plot([mu-np.sqrt(sig2), mu-np.sqrt(sig2)], [0, y_max], '--k')
ax.plot([mu+np.sqrt(sig2), mu+np.sqrt(sig2)], [0, y_max], '--k')

line, = ax.plot([], [], '-', color = 'blue', linewidth=2)

def init():
    '''initialize animation'''
    line.set_data([], [])

    return line, 

def animate(i):
    '''perform animation step''' 
    global rho_I, I_bin
    line.set_data(I_bin, rho_I[:, i])
    
    return line,

ani = animation.FuncAnimation(fig, animate, init_func=init, frames = M, interval = 10, blit = True)
plt.close()

HTML(ani.to_html5_video())

# Bibliographie 

1.  Riksen, H. (1992). The Fokker-Planck Equation. Book (3rd ed., Vol. 11).